In [60]:
import os
from azureml.core import Workspace
from azureml.core import Environment
from azureml.core import ScriptRunConfig, Experiment
from azureml.widgets import RunDetails
from azureml.core import Model

In [61]:
# AMLワークスペースへの接続
ws = Workspace.from_config()

In [62]:
# training script
source_dir = "train"
script_name = "train_bert.py"

# environment file
environment_file = os.path.join("train", "train_bert_env.yml")

# azure ml settings
environment_name = "pl-env-lang"
experiment_name = "bert-livedoor"
compute_name = "shuit-gpu-ins01"

In [63]:
# 学習環境作成、初回のみ長時間
env = Environment.from_conda_specification(environment_name, environment_file)

env.docker.enabled = True
env.docker.base_image = (
    "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04"
)

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [64]:
# 学習設定
src = ScriptRunConfig(
    source_directory=source_dir,
    script=script_name,
    arguments=["--batch_size", 256, "--max_epochs", 20, "--gpus", 1],
    compute_target=compute_name,
    environment=env,
)

In [65]:
# 実行
run = Experiment(ws, experiment_name).submit(src)
run.wait_for_completion(show_output=True)

RunId: bert-livedoor_1621437811_75ba19f4
Web View: https://ml.azure.com/runs/bert-livedoor_1621437811_75ba19f4?wsid=/subscriptions/902f236f-44df-463a-a5cb-1516ab2a9cd2/resourcegroups/shuit-common/workspaces/shuit-ml-workspace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/55_azureml-execution-tvmps_2d354fdb3f2ee253cbc3221283643ba596a25accee557afb6447b7ec996c29a9_d.txt

2021-05-19T15:23:44Z Successfully mounted a/an Blobfuse File System at /mnt/batch/tasks/shared/LS_root/jobs/shuit-ml-workspace/azureml/bert-livedoor_1621437811_75ba19f4/mounts/workspaceblobstore
2021-05-19T15:23:44Z Failed to start nvidia-fabricmanager due to exit status 5 with output Failed to start nvidia-fabricmanager.service: Unit nvidia-fabricmanager.service not found.
. Please ignore this if the GPUs don't utilize NVIDIA® NVLink® switches.
2021-05-19T15:23:44Z Starting output-watcher...
2021-05-19T15:23:44Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-05-19T15:23:44Z Executin

{'runId': 'bert-livedoor_1621437811_75ba19f4',
 'target': 'shuit-gpu-ins01',
 'status': 'Completed',
 'startTimeUtc': '2021-05-19T15:23:44.362247Z',
 'endTimeUtc': '2021-05-19T15:30:24.443439Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'a30f386f-5f2d-49ef-b1ec-9f4d480e6c52',
  'azureml.git.repository_uri': 'https://github.com/ShuntaIto/azureml-pl-sample.git',
  'mlflow.source.git.repoURL': 'https://github.com/ShuntaIto/azureml-pl-sample.git',
  'azureml.git.branch': 'bug-fix',
  'mlflow.source.git.branch': 'bug-fix',
  'azureml.git.commit': '03221e9a689a0094ce957f0f355a6890aef7c80a',
  'mlflow.source.git.commit': '03221e9a689a0094ce957f0f355a6890aef7c80a',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train_bert.py',
  'command': '',
  'useAbsolutePath': False,
  'argument

In [66]:
# モデル登録
run.register_model(
    model_name="bert-livedoor-model",
    model_path=os.path.join('outputs', 'model.onnx'),
    model_framework=Model.Framework.ONNX,
)

Model(workspace=Workspace.create(name='shuit-ml-workspace', subscription_id='902f236f-44df-463a-a5cb-1516ab2a9cd2', resource_group='shuit-common'), name=bert-livedoor-model, id=bert-livedoor-model:4, version=4, tags={}, properties={})